In [ ]:
#how data are represented at each level (forward, backward, forward with padding on top) needs a little
    #experimentation to determine the best representation
    #also, is encoding at each layer really the best way? or just feeding the raw through?
    
#Outside web ips are going to be a problem/messy/noisy. Start by categorizing all outside ips by <OUTSIDE_IP>
    #instead of the ip address, or another 4 digit symbol to insert into the hex string.
    
#to help the models generalize more, for a given source ip address with probability p (say p = 0.1) 
    #use the token <OTHER_MACHINE>
    
#should we remove random parts of the header, i.e. checksum

#should I take out bias for RNNs?

#for the decoder,does the fork encoding need to happen ?
    #do we simply cat the hContext with the next words?
    
#Should the architecture just be encode, context and then prediction???

#Input data, should it have character and hex pair encoding as well?

In [1]:
%matplotlib inline

import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'#,optimizer=fast_compile'

import sys
import binascii
import multiprocessing as mp
from itertools import chain
from scapy.all import *
sys.path.append('hed-dlg/')

import numpy as np
import random
from scipy.stats import itemfreq
import matplotlib.pyplot as plt
from copy import copy

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP, \
                                Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, Fork, LSTM
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T

###These warnings do not impede progress
#WARNING: Failed to execute tcpdump. Check it is installed and in the PATH
#WARNING: No route found for IPv6 destination :: (no default route?)


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


# Data

In [2]:
dataPath = '/data/fs4/datasets/pcaps/smallFlows.pcap'
pcaps = rdpcap(dataPath)
sessionPrep = pcaps.sessions()

In [41]:
import ast
import json
import subprocess
import sys

def parse_header(line):
    ret_dict = {}
    h = line.split()
    #ret_dict['direction'] = " ".join(h[3:6])

    if h[2] == 'IP6':
        """
        Conditional formatting based on ethernet type.
        IPv4 format: 0.0.0.0.port
        IPv6 format (one of many): 0:0:0:0:0:0.port
        """
        ret_dict['src_port'] = h[3].split('.')[-1]
        ret_dict['src_ip'] = h[3].split('.')[0]

        ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
        ret_dict['dest_ip'] = h[5].split('.')[0]
    else:
        if len(h[3].split('.')) > 4:
            ret_dict['src_port'] = h[3].split('.')[-1]
            ret_dict['src_ip'] = '.'.join(h[3].split('.')[:-1])
        else:
            ret_dict['src_ip'] = h[3]
            ret_dict['src_port'] = ''

        if len(h[5].split('.')) > 4:
            ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
            ret_dict['dest_ip'] = '.'.join(h[5].split('.')[:-1])
        else:
            ret_dict['dest_ip'] = h[5].split(':')[0]
            ret_dict['dest_port'] = ''

    try:
        if ret_dict['src_port'] == '53' or ret_dict['dst_port'] == '53':
            ret_dict['length'] = int(h[-1][1:-1])
        else:
            try:
                ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
            except:
                ret_dict['length'] = 0
    except:
        try:
            ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
        except:
            ret_dict['length'] = 0

    return ret_dict

def parse_data(line, length):
    ret_str = ''
    h, d = line.split(':', 1)
    ret_str = d.strip().replace(' ', '')
    if length != 0:
        ret_str = ret_str[:-(2 * length)]
    return ret_str

def process_packet(output):
    # TODO!! throws away the first packet!
    ret_header = {}
    ret_dict = {}
    ret_data = ''
    for line in output:
        line = line.strip()
        if line.startswith('0x'):
            data = parse_data(line, ret_header['length'])
            ret_data = ret_data + data
        else:
            ret_header = parse_header(line)
            ret_dict.update(ret_header)
            if ret_data != '':
                ret_dict['data'] = ret_data
                ret_data = ''
            yield ret_dict

def read_pcap(path):
    hex_sessions = {}
    proc = subprocess.Popen('tcpdump -nn -tttt -xx -r '+path,
                            shell=True,
                            stdout=subprocess.PIPE)
    for packet in process_packet(proc.stdout):
        a = str(packet)
        a = ast.literal_eval(a)
        if 'data' in a:
            key = (a['src_ip']+":"+a['src_port'], a['dest_ip']+":"+a['dest_port'])
            keys = [k for k in hex_sessions if (k[0] == key[0] and k[1] == key[1]) or (k[0] == key[1] and k[1] == key[0])]
            if len(keys) > 0:
                #hex_sessions[keys[0]].append(a['direction']+a['data'])
                hex_sessions[keys[0]].append(a['data'])
            else:
                #hex_sessions[key] = [a['direction']+a['data']]
                hex_sessions[key] = [a['data']]
    return hex_sessions

In [ ]:
hexSessions = read_pcap(dataPath)

In [3]:
#turns the sessions into a dictionary key = session_number, val = list of packages in hex
minPackets = 2

def hexSessionDictCreator(scapySessions, minPackets = 2):

    hexSessions = {}
    i=0
    for k,v in sessionPrep.items(): # v is the session
        
        if len(v) < minPackets:
            pass
        
        else:
            scpcaps = []    
            
            for p in v: #p is the individual packet in the session  
                
                try: #getting rid of payload
                    rawindex = len(p[Raw]) 
                    #payloadLens.append(rawindex)
                    scpcaps.append(binascii.hexlify(str(p.original)[:-rawindex])) #turn it into hex
                
                except: #if no payload
                    scpcaps.append(binascii.hexlify(str(p.original)))
    
            hexSessions['session_' + str(i)] = scpcaps
            i+=1
    #assert that all sessions have len of at least minPackets
    return hexSessions

In [4]:
hexSessions = hexSessionDictCreator(sessionPrep, minPackets=minPackets)

In [5]:
#Making the hex dictionary
def hexTokenizer():
    hexstring = '0,	1,	2,	3,	4,	5,	6,	7,	8,	9,	A,	B,	C,	D,	E,	F,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19\
    ,	1A,	1B,	1C,	1D,	1E,	1F,	20,	21,	22,	23,	24,	25,	26,	27,	28,	29,	2A,	2B,	2C,	2D,	2E,	2F,	30,	31,	32,	33,	34,	35\
    ,	36,	37,	38,	39,	3A,	3B,	3C,	3D,	3E,	3F,	40,	41,	42,	43,	44,	45,	46,	47,	48,	49,	4A,	4B,	4C,	4D,	4E,	4F,	50,	51\
    ,	52,	53,	54,	55,	56,	57,	58,	59,	5A,	5B,	5C,	5D,	5E,	5F,	60,	61,	62,	63,	64,	65,	66,	67,	68,	69,	6A,	6B,	6C,	6D\
    ,	6E,	6F,	70,	71,	72,	73,	74,	75,	76,	77,	78,	79,	7A,	7B,	7C,	7D,	7E,	7F,	80,	81,	82,	83,	84,	85,	86,	87,	88,	89\
    ,	8A,	8B,	8C,	8D,	8E,	8F,	90,	91,	92,	93,	94,	95,	96,	97,	98,	99,	9A,	9B,	9C,	9D,	9E,	9F,	A0,	A1,	A2,	A3,	A4,	A5\
    ,	A6,	A7,	A8,	A9,	AA,	AB,	AC,	AD,	AE,	AF,	B0,	B1,	B2,	B3,	B4,	B5,	B6,	B7,	B8,	B9,	BA,	BB,	BC,	BD,	BE,	BF,	C0,	C1\
    ,	C2,	C3,	C4,	C5,	C6,	C7,	C8,	C9,	CA,	CB,	CC,	CD,	CE,	CF,	D0,	D1,	D2,	D3,	D4,	D5,	D6,	D7,	D8,	D9,	DA,	DB,	DC,	DD\
    ,	DE,	DF,	E0,	E1,	E2,	E3,	E4,	E5,	E6,	E7,	E8,	E9,	EA,	EB,	EC,	ED,	EE,	EF,	F0,	F1,	F2,	F3,	F4,	F5,	F6,	F7,	F8,	F9\
    ,	FA,	FB,	FC,	FD,	FE,	FF'.replace('\t', '')

    hexList = [x.strip() for x in hexstring.lower().split(',')]
    hexList.append('<EOP>') #End Of Packet token
    #EOS token??????
    hexDict = {}

    for key, val in enumerate(hexList):
        if len(val) == 1:
            val = '0'+val
        hexDict[val] = key  #dictionary k=hex, v=int  
    
    return hexDict

In [6]:
hexDict = hexTokenizer()

# Dictionary of IP communications

In [7]:
def srcIpDict(hexSessionDict):
    ''' 
    input: dictionary of key = sessions, value = list of HEX HEADERS of packets in session
    output: dictionary of key = source IP, value/subkey = dictionary of destination IPs, 
                                           subvalue = [[sport], [dport], [plen], [protocol]]
    
    '''
    
    srcIpDict = {}   
    uniqIPs = [] #some ips are dest only. this will collect all ips, not just srcIpDict.keys()
    
    for session in hexSessionDict.keys():
        
        for rawpacket in hexSessionDict[session]:
            packet = copy(rawpacket)
            
            dstIpSubDict = {}

            srcip = packet[52:60]
            dstip = packet[60:68]
            sport = packet[68:72]
            dport = packet[72:76]
            plen = packet[32:36]
            protocol = packet[46:48]
            
            uniqIPs = list(set(uniqIPs) | set([dstip, srcip]))
            
            if srcip not in srcIpDict:
                dstIpSubDict[dstip] = [[sport], [dport], [plen], [protocol]]
                srcIpDict[srcip] = dstIpSubDict
            
            if dstip not in srcIpDict[srcip]:    
                srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
            else:
                srcIpDict[srcip][dstip][0].append(sport)
                srcIpDict[srcip][dstip][1].append(dport)
                srcIpDict[srcip][dstip][2].append(plen)
                srcIpDict[srcip][dstip][3].append(protocol)
                

    return srcIpDict, uniqIPs

In [8]:
def dictUniquerizer(dictOdictsOlistOlists):
    '''
    input: dictionary of dictionaries that have a list of lists 
           ex. srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
    output: dictionary of dictionaries with list of lists with unique items in the final sublist
    
    WARNING: will overwrite your input dictionary. Make a copy if you want to preserve dictOdictsOlistOlists.
    '''
    #dictCopy
    for key in dictOdictsOlistOlists.keys():
        for subkey in dictOdictsOlistOlists[key].keys():
            for sublist in xrange(len(dictOdictsOlistOlists[key][subkey])):
                dictOdictsOlistOlists[key][subkey][sublist] = list(set(dictOdictsOlistOlists[key][subkey][sublist]))
    
    return dictOdictsOlistOlists

In [9]:
#test
comsDict, uniqIPs = srcIpDict(hexSessions)
#test
comsDict = dictUniquerizer(comsDict)

# Adversarial Functions

### Directionality adversary

In [10]:
def ipDirSwitcher(hexSessionList):
    '''
    input is a list of packets from ONE session
    '''
    
    ipdirsession = []
        
    for p in hexSessionList:
        sourceIP = p[52:60]
        destIP = p[60:68]

        ipdirsession.append(p[:52]+destIP+sourceIP+p[68:])

    return ipdirsession

In [11]:
def portDirSwitcher(hexSessionList):
    '''
    input is a list of packets from ONE session
    '''
    
    portdirsession = []
    
    for p in hexSessionList:
        sport = p[68:72]
        dport = p[72:76]

        portdirsession.append(p[:68]+dport+sport+p[76:])

    return portdirsession

### Leave one swap one

In [12]:
def dstIpSwapOut(hexSessionList, dictOcoms, listOuniqIPs):
    #srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
    
    swapSession = []
    srcip = hexSessionList[0][52:60] #assumes first packet contains true initial direction
    dstip = hexSessionList[0][60:68]
    normDstIps = dictOcoms[srcip].keys()+[srcip] #get list of dstIPs that srcIP talks to
    abbynormIps = copy(listOuniqIPs)
    
    for normIp in normDstIps:
        abbynormIps.remove(normIp) #remove itself and know dstIPs from list of consideration.
    
    abbynormDestIp = random.sample(abbynormIps, 1)[0] #get random ip that srcip doesn't talk to

    for rawpacket in hexSessionList:
        packet = copy(rawpacket)
        
        if packet[60:68] == dstip:
            packet = packet[:60] + abbynormDestIp + packet[68:] #
        elif packet[61:69] == srcip:
            packet = packet[:52] + abbynormDestIp + packet[60:] #in case direction switches for packet in session
            
        swapSession.append(packet)

    return swapSession
    

In [ ]:
def dstPortSwapOneOut(hexSessionList):
    
    
    
    
    
    

In [13]:
def oneHot(index, granular = 'hex'):
    if granular == 'hex':
        vecLen = 257
    else:
        vecLen = 17
    
    zeroVec = np.zeros(vecLen)
    zeroVec[index] = 1.0
    
    return zeroVec
 

In [ ]:
def oneSessionTokenized(sessionDict, hexDict, packetTimeSteps=50):

    dstMac = []
    srcMac = []
    firstType = []
    ihl = []
    tos = []
    firLen = []
    firId = []
    frag = []
    ttl = []
    protocol = []
    firCheck = []
    srcip = []
    dstip = []
    sport = []
    dport = []
    
    i=0
    for key in sessionDict.keys():
        sessionPackets = sessionDict[key]
        
        #if len(sessionPackets) > maxPackets: #crop the number of sessions to maxPackets
        #    sessionList = sessionPackets[:maxPackets]
        #else:
        #    sessionList = sessionPackets

        for packet in sessionPackets:
            
            dstMac.append(packet[0:12]) 
            srcMac.append(packet[12:24]) 
            firstType.append(packet[24:28])
            ihl.append(packet[28:30]) 
            tos.append(packet[30:32])
            firLen.append(packet[32:36])
            firId.append(packet[36:40]) 
            frag.append(packet[40:44]) 
            ttl.append(packet[44:46])
            protocol.append(packet[46:48])
            firCheck.append(packet[48:52]) 
            srcip.append(packet[52:60])
            dstip.append(packet[60:68])
            sport.append(packet[68:72])
            dport.append(packet[72:76])
            
            i+=1
    print i
    return np.asarray(dstMac), np.asarray(srcMac),np.asarray(firstType),np.asarray(ihl),np.asarray(tos),\
           np.asarray(firLen),np.asarray(firId),np.asarray(frag),np.asarray(ttl),np.asarray(protocol),\
           np.asarray(firCheck),np.asarray(srcip),np.asarray(dstip), np.asarray(sport), np.asarray(dport)

In [ ]:
netstats  = oneSessionTokenized(hexSessions, hexDict)

In [ ]:
indx = ['dstMac', 'srcMac', 'firstType', 'ihl', 'tos', 'firLen', 'firId', 'frag', 'ttl', 'protocol', 'firCheck',
       'srcip', 'dstip', 'sport', 'dport']
for num in range(13):
    print indx[num], len(np.unique(netstats[num]))


In [ ]:
packet = hexSessions[hexSessions.keys()[1]][1]

In [ ]:
coll = []
for key in hexSessions.keys():
    sessionPackets = hexSessions[key]
    for packet in sessionPackets:
        coll.append(packet[32:36]+packet[44:46]+packet[46:48]+packet[52:60]+packet[60:68]+packet[68:70]+packet[70:72]+
                   packet[72:74])

In [ ]:
np.unique(coll).shape

In [14]:
maxPackets = 2
packetTimeSteps = 16

def oneSessionEncoder(sessionPackets, hexDict, maxPackets = 2, packetTimeSteps = 100,
                       packetReverse = False, charLevel = False, padOldTimeSteps = True):    
            
    sessionCollect = []
    packetCollect = []
    
    if charLevel:
        vecLen = 17
    else:
        vecLen = 257
    
    if len(sessionPackets) > maxPackets: #crop the number of sessions to maxPackets
        sessionList = copy(sessionPackets[:maxPackets])
    else:
        sessionList = copy(sessionPackets)

    for rawpacket in sessionList:
        packet = copy(rawpacket)
        packet = packet[32:36]+packet[44:46]+packet[46:48]+packet[52:60]+packet[60:68]+\
                 packet[68:70]+packet[70:72]+packet[72:74]
        packet = [hexDict[packet[i:i+2]] for i in xrange(0,len(packet)-2+1,2)]
        
        #print np.asarray(packet)
            
        if len(packet) >= packetTimeSteps: #crop packet to length packetTimeSteps
            packet = packet[:packetTimeSteps]
            packet = packet+[256] #add <EOP> end of packet token
        else:
            packet = packet+[256] #add <EOP> end of packet token
        
        packetCollect.append(packet)
        
        pacMat = np.array([oneHot(x) for x in packet]) #one hot encoding of packet into a matrix
        pacMatLen = len(pacMat)
        
        #padding packet
        if packetReverse:
            pacMat = pacMat[::-1]

        if pacMatLen < packetTimeSteps:
            #pad by stacking zeros on top of data so that earlier timesteps do not have information
            #padding the packet such that zeros are after the actual info for better translation
            if padOldTimeSteps:
                pacMat = np.vstack( ( np.zeros((packetTimeSteps-pacMatLen,vecLen)), pacMat) ) 
            else:
                pacMat = np.vstack( (pacMat, np.zeros((packetTimeSteps-pacMatLen,vecLen))) ) 

        if pacMatLen > packetTimeSteps:
            pacMat = pacMat[:packetTimeSteps, :]

        sessionCollect.append(pacMat)

    #padding session
    sessionCollect = np.asarray(sessionCollect, dtype=theano.config.floatX)
    numPacketsInSession = sessionCollect.shape[0]
    if numPacketsInSession < maxPackets:
        #pad sessions to fit the 
        sessionCollect = np.vstack( (sessionCollect,np.zeros((maxPackets-numPacketsInSession, 
                                                             packetTimeSteps, vecLen))) )
    
    return sessionCollect, packetCollect

# Learning functions

In [15]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def dropout(X, p=0.):
    if p != 0:
        retain_prob = 1 - p
        X = X / retain_prob * srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
    return X

# Gradient clipping
def clip_norm(g, c, n): 
    '''n is the norm, c is the threashold, and g is the gradient'''
    
    if c > 0: 
        g = T.switch(T.ge(n, c), g*c/n, g) 
    return g

def clip_norms(gs, c):
    norm = T.sqrt(sum([T.sum(g**2) for g in gs]))
    return [clip_norm(g, c, norm) for g in gs]

# Regularizers
def max_norm(p, maxnorm = 0.):
    if maxnorm > 0:
        norms = T.sqrt(T.sum(T.sqr(p), axis=0))
        desired = T.clip(norms, 0, maxnorm)
        p = p * (desired/ (1e-7 + norms))
    return p

def gradient_regularize(p, g, l1 = 0., l2 = 0.):
    g += p * l2
    g += T.sgn(p) * l1
    return g

def weight_regularize(p, maxnorm = 0.):
    p = max_norm(p, maxnorm)
    return p

def Adam(params, cost, lr=0.0002, b1=0.1, b2=0.001, e=1e-8, l1 = 0., l2 = 0., maxnorm = 0., c = 8):
    
    updates = []
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    
    i = theano.shared(floatX(0.))
    i_t = i + 1.
    fix1 = 1. - b1**(i_t)
    fix2 = 1. - b2**(i_t)
    lr_t = lr * (T.sqrt(fix2) / fix1)
    
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        g_t = gradient_regularize(p, g_t, l1=l1, l2=l2)
        p_t = p - (lr_t * g_t)
        p_t = weight_regularize(p_t, maxnorm=maxnorm)
        
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    
    updates.append((i, i_t))
    return updates

def RMSprop(cost, params, lr = 0.001, l1 = 0., l2 = 0., maxnorm = 0., rho=0.9, epsilon=1e-6, c = 8):
    
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    updates = []
    
    for p, g in zip(params, grads):
        g = gradient_regularize(p, g, l1 = l1, l2 = l2)
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        updates.append((acc, acc_new))
        
        updated_p = p - lr * (g / T.sqrt(acc_new + epsilon))
        updated_p = weight_regularize(updated_p, maxnorm = maxnorm)
        updates.append((p, updated_p))
    return updates

In [ ]:
#makes output by shifting inputs down in time one step and then copying the last time step to the end.
#def targetModifier(targetArray):
#    newTarget = np.vstack((targetArray[1:, :], targetArray[-1,:]))
#    return newTarget

#def targetMaker(listOinputs):
    #TODO: do this with arrays
#    outputs = []
#    for inp in listOinputs:
#        outputs.append(targetModifier(inp))
#    outputs = np.asarray(outputs)
#    
#    return outputs

# Unsupervised feature extractor

### Initialization for both the unsupervised net and the classifier

In [86]:
X = T.tensor4('inputs')
Y = T.matrix('targets')

dimIn = 257 #hex has 256 characters + the <EOP> character
dim = 100 #dimension reduction size
rnnType = 'gru' #gru or lstm
bidirectional = False
linewt_init = IsotropicGaussian(0.1)
line_bias = Constant(1.0)
rnnwt_init = IsotropicGaussian(0.1)
rnnbias_init = Constant(0.0)
packetReverse = False

###ENCODER
if rnnType == 'gru':
    rnn = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIn, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)


###CONTEXT
if rnnType == 'gru':
    rnnContext = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                biases_init = rnnbias_init, name = 'gruContext')
else:
    rnnContext = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                      name = 'lstmContext')

forkContext = Fork(output_names=['linearContext', 'gatesContext'],
            name='forkContext', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

if bidirectional:
    dimDec = dim*2
    
    if rnnType == 'gru':
        rnnContextRev = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                       biases_init = rnnbias_init, name = 'gruContextRev')
        
    else:
        rnnContextRev = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init,
                             name = 'lstmContextRev')
    
    rnnContextRev.initialize()

else:
    dimDec = dim


###DECODER
'''if rnnType == 'gru':
    rnnDec = GatedRecurrent(dim=dimIn, weights_init = rnnwt_init, 
                            biases_init = rnnbias_init, name = 'gruDecoder')
else:
    rnnDec = LSTM(dim=dimIn, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstmDecoder')'''


forkDec = Fork(output_names=['linear', 'gates'],
            name='forkDec', input_dim=dimDec, output_dims=[dim, dim*dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

#forkFinal = Fork(output_names=['linear', 'gates'],
#            name='forkFinal', input_dim=dim, output_dims=[dimIn, dimIn*dimMultiplier], 
#            weights_init = linewt_init, biases_init = line_bias)

#initialize the weights in all the functions
fork.initialize()
rnn.initialize()

forkContext.initialize()
rnnContext.initialize()
forkDec.initialize()

#forkFinal.initialize()
#rnnDec.initialize()

### Unsupervised graph

In [ ]:
#This section can be skipped if you want to go directly to the classifier

batch_size = 20

data1, data2 = fork.apply(X) #data1 shape = (batch_size, maxPackets, dimIn)

if rnnType == 'gru':
    hEnc = rnn.apply(data1, data2)[:,-1] #the [:,-1] gets the last hidden state for each obs in minibatch
                                         #i.e. the last state for each sentence
else:
    hinit, _ = rnn.apply(data2)
    hEnc = hinit[:,-1] #hEnc shape = (batch_size*maxPackets, dim) 

hEnc = T.reshape(hEnc,(batch_size, maxPackets, dim))

data3, data4 = forkContext.apply(hEnc)

if rnnType == 'gru':
    hContext = rnnContext.apply(data3, data4)
else:
    hinitContext, _ = rnnContext.apply(data4)
    hContext = hinitContext #hContext shape = (batch_size, maxPackets, dim)

#TODO:test bidirectional and make work

'''if bidirectional:
    data3 = data3[::-1]
    data4 = data4[::-1]

    if rnnType == 'gru':
        hContextRev = rnnContextRev.apply(data3, data4)
    else:
        hinitContext, _ = rnnContextRev.apply(data4)
        hContextRev = hinitContext

    hContext = T.concatenate((hContext, hContextRev), axis=2)
'''

data5, _ = forkDec.apply(hContext) #this fork makes data5 same dim as data1 (the orig word embedding)

#decoding data needs to be one timestep (next packet in session) ahead, thus data1 we ignore the first packet
#and the last hidden state of the context RNN.
#THINK about L2 pooling before cat
#THINK should we concatenate with X instead of data5
#if packetReverse:
#    data1 = data1[:,::-1]

data7 = T.concatenate((T.reshape(data5[:,:-1],(batch_size*(maxPackets-1),1,dim)), 
                       T.reshape(T.reshape(data1,(batch_size, maxPackets, packetTimeSteps, dim))[:,1:,1:,:], 
                                 (batch_size*(maxPackets-1),packetTimeSteps - 1,dim))), axis=1) 
                      
                      #data1 is the original embedding of X, data5 is transformed context output
                      #get rid of first packet in data 5
                      #get rid of last context vector
                      
data8, data9 = forkFinal.apply(data7) #forkFinal transforms back to original dimIn

if rnnType == 'gru':
    hDec = rnnDec.apply(data8, data9) 
else:
    hinit, _ = rnnDec.apply(data9)
    hDec = hinit #hDec shape = (batch_size*(maxPackets-1), packetTimeSteps, 257)

softmax = NDimensionalSoftmax()
softout = softmax.apply(hDec, extra_ndim = 1)
predX = T.reshape(T.reshape(X,(batch_size, maxPackets, packetTimeSteps, dimIn))[:,1:,:,:], 
                  (batch_size*(maxPackets-1), packetTimeSteps, 257))

precost = predX*T.log(softout) + (1-predX)*T.log(1-softout)
precost2 = -T.sum(T.sum(precost, axis = 2), axis = 1)
#precost2 = -T.mean(T.sum(T.sum(precost, axis = 2), axis = 1))

#cost = T.mean(precost2)
cost = T.mean(BinaryCrossEntropy().apply(predX, softout))
cg = ComputationGraph([cost])
learning_rate = 0.0001
params = VariableFilter(roles = [PARAMETER])(cg.variables)
updates = Adam(params, cost, learning_rate, c=5) #c is gradient clipping parameter
#updates = RMSprop(cost, params, learning_rate, c=5)

#gradients = T.grad(cost, params)
#gradients = clip_norms(gradients, 1)
#gradientFun = theano.function([X], gradients, allow_input_downcast=True)

print "compiling you beautiful person"
train = theano.function([X], [cost, hContext], updates = updates, allow_input_downcast=True)
predict = theano.function([X], softout, allow_input_downcast=True)
print "finished compiling"

### Unsupervised training

In [ ]:
#randomize data
hexSessionsKeys = hexSessions.keys()
random.shuffle(hexSessionsKeys)
trainPercent = 0.9
trainIndex = int(len(hexSessionsKeys)*trainPercent)

runname = 'hred'
epochCost = []
gradNorms = []
contextCollect = []

epochs = 10
iteration = 0

for epoch in xrange(epochs):
    costCollect = []
    
    for start, end in zip(range(0, trainIndex,batch_size), range(batch_size, trainIndex, batch_size)):
        
        trainingSessions = []
        
        for trainKey in range(start, end):
            sessionForEncoding = hexSessions[hexSessions.keys()[trainKey]]
            oneHotSes = oneSessionEncoder(sessionForEncoding, packetReverse=packetReverse, 
                                          padOldTimeSteps = packetReverse,
                                          hexDict = hexDict,
                                          maxPackets = maxPackets, packetTimeSteps = packetTimeSteps)
            trainingSessions.append(oneHotSes[0])
        
        trainingSessions = [item for sublist in trainingSessions for item in sublist]
        sessionsMinibatch = np.asarray(trainingSessions)
        
    
        costfun = train(sessionsMinibatch)
        costCollect.append(costfun[0])
        
            
        iteration+=1
        
        '''if iteration%20:
            print '   iteration: ', iteration
            print '   intermed cost: ', np.mean(costCollect)'''
    ####SAVE COST TO FILE  
    if epoch%2 == 0:
        print(' ')
        print 'Epoch: ', epoch
        epochCost.append(np.mean(costCollect))
        contextCollect.append(costfun[1][:4])
        print 'Epoch cost average: ', epochCost[-1]
        #grads = gradientFun(inputs, outputs)
        #for gra in grads:
        #    print '  gradient norms: ', np.linalg.norm(gra)
        
    np.savetxt(runname+"_COST.csv", epochCost, delimiter=",")

# Classifier

### Classifier graph

In [87]:
#Can be used with or without unsupervised learning

#X = T.matrix('classtest')
#Y = T.matrix('targets')
batch_sizeClass = 20
numClasses = 2
clippings = 1
learning_rateClass = 0.01
classifierWts = IsotropicGaussian(0.1, 0)

bmlp = BatchNormalizedMLP(activations=[Logistic(),Logistic()], 
           dims=[dim, dim, numClasses],
           weights_init=classifierWts,
           biases_init=Constant(0.0001) )


bmlp.initialize()

def onestepEnc(X):
    data1, data2 = fork.apply(X) #data1 shape = (batch_size, maxPackets, dimIn)

    if rnnType == 'gru':
        hEnc = rnn.apply(data1, data2) 
    else:
        hEnc, _ = rnn.apply(data2)
        #hinit, _ = rnn.apply(data2)
        #hEnc = hinit[:,-1] #hEnc shape = (batch_size, maxPackets, dim) 

    #hEnc2 = hEnc[-1]#T.reshape(hEnc[:,-1],(batch_sizeClass, maxPackets, dim))#the [:,-1] gets the last hidden state for 
                                                                    #each obs in minibatch
                                                                    #i.e. the last state for each sentence
    return hEnc

hEnc, _ = theano.scan(onestepEnc, X)
hEncReshape = T.reshape(hEnc[:,-1], (batch_sizeClass,maxPackets,1,dim))

testenc = theano.function([X], hContextReshape, allow_input_downcast=True)

testenc(np.ones((40,2,1,257))).shape

In [88]:
def onestepContext(hEncReshape):
    
    data3, data4 = forkContext.apply(hEncReshape)
    
    if rnnType == 'gru':
        hContext = rnnContext.apply(data3, data4)
    else:
        hinitContext, _ = rnnContext.apply(data4)
        hContext = hinitContext #hContext shape = (batch_size, maxPackets, dim)
    
    return hContext

In [89]:
hContext, _ = theano.scan(onestepContext, hEncReshape)
hContextReshape = T.reshape(hContext[:,-1], (batch_sizeClass,dim))

In [90]:
data5, _ = forkDec.apply(hContextReshape)

pyx = bmlp.apply(data5)
softmax = Softmax()
softoutClass = softmax.apply(pyx)
costClass = T.mean(CategoricalCrossEntropy().apply(Y, softoutClass))

cgClass = ComputationGraph([costClass])
paramsClass = VariableFilter(roles = [PARAMETER])(cgClass.variables)

#updatesClass = Adam(paramsClass, costClass, learning_rateClass, c=clippings) 
updatesClass = RMSprop(costClass, paramsClass, learning_rateClass, c=clippings)

print 'grad compiling'
gradients = T.grad(costClass, paramsClass)
gradients = clip_norms(gradients, clippings)
gradientFun = theano.function([X,Y], gradients, allow_input_downcast=True)
print 'finish with grads'

print 'compiling functions you talented soul'
classifierTrain = theano.function([X,Y], [costClass, hEnc, hContext,pyx, softoutClass], updates=updatesClass, allow_input_downcast=True)
classifierPredict = theano.function([X], softoutClass, allow_input_downcast=True)
print 'finished compiling'

grad compiling
finish with grads
compiling functions you talented soul
finished compiling


In [67]:
testfun = theano.function([X], data5, allow_input_downcast=True)

In [59]:
np.mean(np.argmax(testfun(sessionsMinibatch), axis = 1) == targetsMinibatch[:,1])

0.65000000000000002

In [68]:
testfun(sessionsMinibatch)

array([[ -4.08552187e-05,   6.52331073e-05,  -2.25954122e-04,
         -1.36082061e-04,  -7.98002293e-06,   8.73750178e-05,
          2.70325472e-05,   1.24739963e-04,   3.48383648e-04,
         -1.28321408e-04,   1.09236425e-04,  -3.02765184e-05,
         -2.40783629e-05,  -1.76583038e-04,  -8.73717727e-06,
         -2.33404600e-04,   2.26700911e-04,   7.02253965e-05,
         -4.26690895e-05,   1.10909677e-04,   1.87109967e-04,
         -8.72958917e-05,  -3.27984453e-05,  -1.36528979e-04,
          2.00399983e-04,   1.63435412e-04,   4.90299317e-06,
          1.39666867e-04,   2.22454291e-05,   4.94174783e-05,
          1.50146312e-04,   2.91250144e-05,   5.92990509e-05,
          1.95969897e-05,  -6.95251802e-05,  -3.13746968e-05,
         -1.18822209e-04,   1.64653247e-04,   3.89593260e-05,
          2.20949805e-05,  -8.09883350e-05,  -1.47104874e-04,
         -2.93077930e-04,   8.31233683e-06,   2.04733806e-06,
          2.28485587e-05,   4.18248746e-05,   3.89189154e-06,
        

### Classifier training

In [91]:
#randomize data, if not already done in unsupervised portion
hexSessionsKeys = hexSessions.keys()
#random.shuffle(hexSessionsKeys)
trainPercent = 0.9
trainIndex = int(len(hexSessionsKeys)*trainPercent)
packetReverse = False
padOldTimeSteps = True

runname = 'hredClassify'
epochCost = []
gradNorms = []

epochs = 1000
iteration = 0

normalTarget = np.array([0,1], dtype=theano.config.floatX)
abbyTarget = np.array([1,0], dtype=theano.config.floatX)

for epoch in xrange(epochs):
    costCollect = []

    for start, end in zip(range(0, trainIndex,batch_sizeClass/2),
                          range(batch_sizeClass/2, trainIndex, batch_sizeClass/2)):

        #build a 4d array of oneHot sessions
        
        trainingTargets = []
        trainingSessions = []
        for trainKey in range(start, end):
            sessionForEncoding = list(hexSessions[hexSessions.keys()[trainKey]])
            
            #encode a normal session
            oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
                                          padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
                                          packetTimeSteps = packetTimeSteps)
            trainingSessions.append(oneHotSes[0])
            trainingTargets.append(normalTarget)
            
            #encode an abby normal session
            abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs)
            #abbyHexSession = portDirSwitcher(sessionForEncoding)
            abbyOneHotSes = oneSessionEncoder(abbyHexSession,hexDict = hexDict,packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)
            trainingSessions.append(abbyOneHotSes[0])
            trainingTargets.append(abbyTarget)
            
        #trainingSessions2 = [item for sublist in trainingSessions for item in sublist] #FIX
        sessionsMinibatch = np.asarray(trainingSessions).reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, 257))
        targetsMinibatch = np.asarray(trainingTargets)
    
        costfun = classifierTrain(sessionsMinibatch, targetsMinibatch)
        costCollect.append(costfun[0])
                
        iteration+=1
        
    ####SAVE COST TO FILE  
    if epoch%2 == 0:
        print(' ')
        print 'Epoch: ', epoch
        epochCost.append(np.mean(costCollect))
        print 'Epoch cost average: ', epochCost[-1]
        grads = gradientFun(sessionsMinibatch, targetsMinibatch)
        print 'train accuracy: ', np.mean(np.argmax(costfun[-1],axis=1) == targetsMinibatch[:,1])
        print costfun[-1]
        for gra in grads:
            print '  gradient norms: ', np.linalg.norm(gra)
        
    np.savetxt(runname+"_COST.csv", epochCost, delimiter=",")

 
Epoch:  0
Epoch cost average:  0.694072
train accuracy:  0.5
[[ 0.50013024  0.49986985]
 [ 0.50011772  0.49988228]
 [ 0.50013918  0.49986085]
 [ 0.50015932  0.49984074]
 [ 0.50017017  0.49982983]
 [ 0.50016433  0.49983567]
 [ 0.50013524  0.49986476]
 [ 0.50014669  0.49985328]
 [ 0.5001694   0.49983054]
 [ 0.5001514   0.49984866]
 [ 0.50023353  0.49976653]
 [ 0.50024313  0.4997569 ]
 [ 0.50015199  0.49984798]
 [ 0.5001744   0.49982566]
 [ 0.50015932  0.49984074]
 [ 0.50017303  0.49982703]
 [ 0.50014555  0.49985442]
 [ 0.5001691   0.4998309 ]
 [ 0.5001353   0.49986473]
 [ 0.50011939  0.49988055]]
  gradient norms:  2.55651e-05
  gradient norms:  5.50307e-05
  gradient norms:  0.000137269
  gradient norms:  3.41082e-06
  gradient norms:  4.39799e-06
  gradient norms:  5.31536e-05
  gradient norms:  3.01433e-06
  gradient norms:  3.3706e-05
  gradient norms:  1.10027e-05
  gradient norms:  3.04262e-06
  gradient norms:  1.6702e-06
  gradient norms:  7.85756e-07
  gradient norms:  9.51295

KeyboardInterrupt: 

In [ ]:
#[costClass,hEnc, hContext,pyx]
costfun[1][0][0] == costfun[1][1][0]

In [93]:
#testing for generalization

testCollect = []
for start, end in zip(range(trainIndex, len(hexSessionsKeys), batch_sizeClass/2),
                      range(trainIndex + batch_sizeClass/2, len(hexSessionsKeys), batch_sizeClass/2)):
    trainingTargets = []
    trainingSessions = []
    for trainKey in range(start, end):
        sessionForEncoding = hexSessions[hexSessions.keys()[trainKey]]

        #encode a normal session
        oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
                                      padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
                                      packetTimeSteps = packetTimeSteps)
        trainingSessions.append(oneHotSes[0])
        trainingTargets.append(normalTarget)

        #encode an abby normal session
        abbyHexSession = portDirSwitcher(sessionForEncoding)
        abbyOneHotSes = oneSessionEncoder(abbyHexSession,hexDict = hexDict,packetReverse=packetReverse, 
                                          padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets, 
                                          packetTimeSteps = packetTimeSteps)
        trainingSessions.append(abbyOneHotSes[0])
        trainingTargets.append(abbyTarget)

    #trainingSessions = [item for sublist in trainingSessions for item in sublist] #FIX in oneSessionEncoder
    sessionsMinibatch = np.asarray(trainingSessions, dtype=theano.config.floatX).reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, 257))
    targetsMinibatch = np.asarray(trainingTargets, dtype=theano.config.floatX)

    costfun = classifierPredict(sessionsMinibatch)
    testCollect.append(np.mean(np.argmax(costfun, axis=1) == targetsMinibatch[:,1]))

print 'test accuracy: ', np.mean(testCollect)
    


test accuracy:  0.51


In [98]:
classifierPredict(sessionsMinibatch[::-1])

array([[ 0.26896736,  0.73103261],
       [ 0.26896733,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.26896733,  0.73103267],
       [ 0.73093122,  0.26906878],
       [ 0.73093146,  0.26906857],
       [ 0.7309261 ,  0.26907387],
       [ 0.73087281,  0.26912722],
       [ 0.26896733,  0.73103261],
       [ 0.26896733,  0.73103267],
       [ 0.73093158,  0.26906839],
       [ 0.73093122,  0.26906881],
       [ 0.26896733,  0.73103261],
       [ 0.26896733,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.73093152,  0.26906848],
       [ 0.73092753,  0.2690725 ],
       [ 0.26896742,  0.73103261],
       [ 0.2689673 ,  0.73103267]], dtype=float32)

In [97]:
classifierPredict(sessionsMinibatch)

array([[ 0.2689673 ,  0.73103267],
       [ 0.26896742,  0.73103261],
       [ 0.73092759,  0.26907244],
       [ 0.73093152,  0.26906848],
       [ 0.2689673 ,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.26896733,  0.73103267],
       [ 0.26896733,  0.73103261],
       [ 0.73093122,  0.26906881],
       [ 0.73093158,  0.26906839],
       [ 0.26896733,  0.73103267],
       [ 0.26896733,  0.73103261],
       [ 0.73087281,  0.26912722],
       [ 0.7309261 ,  0.26907387],
       [ 0.73093146,  0.26906857],
       [ 0.73093122,  0.26906878],
       [ 0.26896733,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.2689673 ,  0.73103267],
       [ 0.26896736,  0.73103261]], dtype=float32)

In [92]:
#padOldTimeSteps = True

for trainKey in range(0, 20):
    sessionForEncoding = hexSessions[hexSessions.keys()[trainKey]]
    #encode a normal session
    oneHotSes = oneSessionEncoder(portDirSwitcher(sessionForEncoding),hexDict = hexDict, packetReverse=packetReverse, 
                                  padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
                                  packetTimeSteps = packetTimeSteps)
    trainingSessions.append(oneHotSes[0])
    trainingTargets.append(normalTarget)
    
    #encode an abby normal session
    abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs)
    #abbyHexSession = portDirSwitcher(sessionForEncoding)
    abbyOneHotSes = oneSessionEncoder(abbyHexSession,hexDict = hexDict, packetReverse=packetReverse, 
                                      padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets, 
                                      packetTimeSteps = packetTimeSteps)
    trainingSessions.append(abbyOneHotSes[0])
    trainingTargets.append(abbyTarget)
    
#trainingSessions2 = [item for sublist in trainingSessions for item in sublist] #FIX
sessionsMinibatch = np.asarray(trainingSessions).reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, 257))
targetsMinibatch = np.asarray(trainingTargets)


ValueError: total size of new array must be unchanged

#Can be used with or without unsupervised learning

#X = T.matrix('classtest')
#Y = T.matrix('targets')
batch_sizeClass = 20
numClasses = 2
clippings = 5
learning_rateClass = 0.0001
classifierWts = IsotropicGaussian(0.03, 0)

bmlp = BatchNormalizedMLP(activations=[Logistic(),Logistic()], 
           dims=[dim, dim, numClasses],
           weights_init=classifierWts,
           biases_init=Constant(0.0001) )


bmlp.initialize()

data1, data2 = fork.apply(X) #data1 shape = (batch_size, maxPackets, dimIn)

if rnnType == 'gru':
    hEnc = rnn.apply(data1, data2) 
else:
    hEnc, _ = rnn.apply(data2, iterate = False)
    #hinit, _ = rnn.apply(data2)
    #hEnc = hinit[:,-1] #hEnc shape = (batch_size, maxPackets, dim) 

hEnc2 = T.reshape(hEnc[-1],(batch_sizeClass, maxPackets, dim))#the [:,-1] gets the last hidden state for 
                                                                #each obs in minibatch
                                                                #i.e. the last state for each sentence

data3, data4 = forkContext.apply(hEnc2)

if rnnType == 'gru':
    hContext = rnnContext.apply(data3, data4)
else:
    hinitContext, _ = rnnContext.apply(data4)
    hContext = hinitContext #hContext shape = (batch_size, maxPackets, dim)

data5, _ = forkDec.apply(hContext) 

#FIX bidirectional
'''if bidirectional:

    data3classRev = data3class[::-1]
    data4classRev = data4class[::-1]

    if rnnType == 'gru':
        hContextRev = rnnContextRev.apply(data3classRev, data4classRev)
    else:
        hinitContextRevClass, _ = rnnContextRev.apply(data4classRev)
        hContextRevClass = hinitContextRevClass

    hContextClass = T.concatenate((hContextClass, hContextRevClass), axis=2)'''
        

pyx = bmlp.apply(hContext[:,-1])
#softmax = Softmax()
#softoutClass = softmax.apply(pyx)
costClass = T.mean(CategoricalCrossEntropy().apply(Y, pyx))

cgClass = ComputationGraph([costClass])
paramsClass = VariableFilter(roles = [PARAMETER])(cgClass.variables)

updatesClass = Adam(paramsClass, costClass, learning_rateClass, c=clippings) 
#updatesClass = RMSprop(costClass, paramsClass, learning_rateClass, c=clippings)

#print 'grad compiling'
#gradients = T.grad(costClass, paramsClass)
#gradients = clip_norms(gradients, clippings)
#gradientFun = theano.function([X,Y], gradients, allow_input_downcast=True)
#print 'finish with grads'

print 'compiling functions you talented soul'
#classifierTrain = theano.function([X,Y], [costClass,hEnc, hEnc2, hContext,pyx], updates=updatesClass, allow_input_downcast=True)
#classifierPredict = theano.function([X], pyx, allow_input_downcast=True)
print 'finished compiling'